In [1]:
import csv
import functions as func
import Bio.motifs as motifs

In [2]:
Promoter_List = []
with open("Inputs.csv") as Promoter_Data:
    Promoter_Reader = csv.DictReader(Promoter_Data, delimiter=';')
    Temporary_List = ["","","","","","",""]
    Hash_Tracker = [] 
    for Row in Promoter_Reader: 
        Temporary_List[0] = Row["Reference Cell Line"] if Temporary_List[0] == "" else Temporary_List[0]     
        if Row["Hash"] in Hash_Tracker:
            continue;
        Temporary_List[4] = Row["Reference Gene Name"]
        Temporary_List[5] = Row["Reference Sequence"]
        Temporary_List[6] = Row["Hash"]
        if Row["Pathway"] != "":
            Temporary_List[1] = Row["Pathway"]
        if Row["Reaction"] != "":
            Temporary_List[2] = Row["Reaction"]
        if Row["Reference Enzyme"] != "":
            Temporary_List[3] = Row["Reference Enzyme"]
        Promoter_List.append([Temporary_List[0],Temporary_List[1],Temporary_List[2],
                              Temporary_List[3],Temporary_List[4],Temporary_List[5],Temporary_List[6]])
        Hash_Tracker.append(Row["Hash"])

In [3]:
databaseDict = {}
pssmDict = {}
coreDict = {}
with open("databaseFull.txt") as database:
    for Motif in motifs.parse(database, "Jaspar"):
            PWM = Motif.counts.normalize(pseudocounts=func.Pseudocount_Adder(Motif.counts))
            PSSM = PWM.log_odds(background={"A":0.31,"C":0.19, "G":0.19, "T":0.31})
            CORE = func.core_Pssm(Motif.counts, background={"A":0.31,"C":0.19, "G":0.19, "T":0.31})
            
            try:
                databaseDict[Motif.name].append(Motif.base_id)
                pssmDict[Motif.base_id] = PSSM
                coreDict[Motif.base_id] = CORE
            except KeyError:
                databaseDict[Motif.name] = [Motif.base_id]
                pssmDict[Motif.base_id] = PSSM
                coreDict[Motif.base_id] = CORE

In [4]:
tfInteractsWith = {}
with open("Interactions.txt") as InteractionsList:
    for line in InteractionsList:
        if line.startswith("["):
            lineListed = line[1:-2].split(",")
            for index, item in enumerate(lineListed):
                item = item.replace(" ","",-1)
                item = item.replace("\'","",-1)
                lineListed[index] = item
            tfInteractsWith[currentFactor] = [] if lineListed == [''] else lineListed
        else:
            currentFactor = line[:-1]

In [5]:
for key in tfInteractsWith:
    print("{}:\t{}".format(key, len(tfInteractsWith[key])))

F$ABF1_01:	45
F$ABF1_02:	0
F$ABF1_03:	38
F$ABF1_C:	41
F$ABF1_Q6:	0
F$ABF2_01:	15
F$ACE2_01:	12
F$ADR1_01:	36
F$ADR1_02:	37
F$ADR1_Q6:	16
F$RCS1_01:	30
F$RCS1_Q2:	11
F$AFT2_01:	36
F$ARGRI_01:	27
F$ARGRII_01:	26
F$ARO80_01:	30
F$ARR1_01:	18
F$ASG1_01:	0
F$ASG1_02:	43
F$ASH1_01:	35
F$AZF1_01:	36
F$BAS1_01:	28
F$BAS1_Q2:	23
F$CAT8_01:	43
F$CAT8_Q6:	33
F$CAT8_Q6_01:	34
F$CBF1_01:	23
F$CBF1_02:	0
F$CBF1_03:	26
F$CBF1_04:	17
F$CBF1_05:	33
F$CBF1_B:	17
F$CBF1_Q6:	41
F$CEP3_01:	30
F$CEP3_02:	21
F$CHA4_01:	50
F$CHA4_02:	49
F$CIN5_01:	29
F$CRZ1_01:	44
F$CST6_01:	20
F$CUP2_01:	37
F$CUP2_Q4:	43
F$CUP9_01:	6
F$CUP9_02:	17
F$DAL80_01:	15
F$DAL81_01:	0
F$DAL82_01:	33
F$DOT6_01:	36
F$DOT6_02:	0
F$ECM22_01:	42
F$ECM22_02:	39
F$ECM23_01:	17
F$EDS1_01:	41
F$YBR239C_01:	34
F$YBR239C_02:	33
F$FHL1_01:	28
F$FHL1_02:	25
F$FKH1P_02:	0
F$FKH1_01:	30
F$FKH2P_01:	32
F$FKH2P_02:	0
F$FKH2_01:	32
F$FKH2_02:	25
F$FLP1_01:	0
F$FLP1_02:	0
F$FZF1_01:	0
F$GAL4_01:	40
F$GAL4_02:	7
F$GAL4_C:	40
F$GAT1_01:	25
F$GAT1_02:	22


In [6]:
truthTable_Binding = {}
with open("FactorTruthTable_Binding.txt") as aimDatabase:
    for line in aimDatabase:
        lineListed = line.split()
        if len(lineListed) == 0 or lineListed[0] == "Genes" or lineListed[0] == "Not":
            continue
        elif lineListed[-1] == "Promoter" and len(lineListed) > 3:
                truthTable_Binding[lineListed[0]] = []
        elif lineListed[-1] == "to":
                truthTable_Binding[lineListed[0]] = []
                currentKey = lineListed[0]
        elif lineListed[-1] == "found!":
            truthTable_Binding[lineListed[0]] = []
        elif lineListed[-1] == "Reference":
            truthTable_Binding[currentKey].append(lineListed[0])

In [7]:
for key in truthTable_Binding:
    print("{}:\t{}".format(key, len(truthTable_Binding[key])))

Abf1p:	8
Ace2p:	2
Adr1p:	17
Aft1p:	6
Aft2p:	1
Arg80p:	0
Arg81p:	0
Aro80p:	1
Arr1p:	0
Ash1p:	3
Azf1p:	0
Bas1p:	0
Cat8p:	8
Cbf1p:	12
Cha4p:	0
Cin5p:	22
Crz1p:	10
Cst6p:	3
Cup2p:	0
Cup9p:	1
Dal80p:	0
Dal81p:	2
Dal82p:	0
Ecm22p:	0
Ert1p:	7
Fhl1p:	19
Fkh1p:	31
Fkh2p:	28
Fzf1p:	0
Gal4p:	0
Gat1p:	1
Gcn4p:	12
Gcr1p:	10
Gcr2p:	12
Gis1p:	0
Gln3p:	1
Gsm1p:	1
Gzf3p:	1
Hac1p:	0
Hal9p:	1
Hap1p:	10
Hap2p:	1
Hap3p:	1
Hap4p:	4
Hcm1p:	3
Hsf1p:	6
Ifh1p:	5
Ime1p:	0
Ino2p:	21
Ino4p:	17
Ixr1p:	9
Leu3p:	0
Lys14p:	0
Mac1p:	0
Mal63p:	0
Mbp1p:	3
Mcm1p:	3
Met31p:	0
Met32p:	10
Met4p:	1
Mga1p:	2
Mig1p:	4
Mig2p:	0
Mig3p:	0
Mot2p:	1
Mot3p:	1
Msn2p:	40
Msn4p:	37
Ndt80p:	35
Nrg1p:	3
Oaf1p:	18
Pdr1p:	3
Pdr3p:	0
Pdr8p:	0
Phd1p:	8
Pho2p:	4
Pho4p:	9
Put3p:	1
Rap1p:	29
Rds1p:	2
Rds2p:	14
Reb1p:	9
Rfx1p:	5
Rgm1p:	2
Rgt1p:	1
Rim101p:	0
Rlm1p:	0
Rme1p:	0
Rox1p:	2
Ume6p:	2
Rph1p:	1
Rpn4p:	2
Rtg3p:	4
Sfl1p:	0
Sfp1p:	2
Sip4p:	6
Skn7p:	14
Smp1p:	0
Sok2p:	18
Spt23p:	33
Stb4p:	1
Stb5p:	9
Ste12p:	38
Stp1p:	2
Stp2p:	2
Sum1p:	1
Swi4p:

In [8]:
noYeastract = ["ABF2", "ASG1", "CEP3", "DOT6", "ECM23", "EDS1",
                "FLP1", "GAT3", "GAT4", "HMO1", "HMRA1", "HMRA2",
                "MATALPHA2", "MATALPHA2P", "MET28", "STRE", "NHP10",
                "NHP6A", "NHP6B", "OPI1", "POT1P", "RDR1", "REI1",
                "RSC3",	"RSC30", "ZMS1", "TBP", "SRD1", "STB3",
                "SPT2", "STP3", "STP4", "STUAP", "SUT1", "SUT2",
                "CCBF", "TAF", "TBS1", "PBF1", "SGC1", "YDR520C",
                "YGR067C", "YKL222C", "YLL054C", "YLR278C", "YNR063W",
                "YPR013C", "YPR015C", "YPR022C", "YPR196W", "YRM1"]

keyMatched = []
for keyTransfac in databaseDict.keys():
    if keyTransfac not in noYeastract:
        try:
            if keyTransfac[:-1] in keyMatched[-1] and (keyTransfac[-1].isnumeric() == False or keyMatched[-1][-1].isnumeric() == False):
                if keyTransfac[:-1] == "ARGRI":
                    keyMatched.append(keyTransfac)
                    continue    
                string = keyMatched[-1]
                keyMatched[-1] = []
                keyMatched[-1].append(string)
                keyMatched[-1].append(keyTransfac)
            elif keyTransfac == "HAP234":
                keyMatched.append("HAP234")
                keyMatched.append("HAP234")
                keyMatched.append("HAP234")
            else:
                keyMatched.append(keyTransfac)
        except:
            keyMatched.append(keyTransfac)
    else:
        continue

yeastractTOtransfac = {}
for keyTransfac, keyYeastract in zip(keyMatched, truthTable_Binding):
    yeastractTOtransfac[keyYeastract] = keyTransfac

In [9]:
resultCheck = {}
for key in truthTable_Binding:
    resultCheck[key] = {}
    
    transfacVersion = yeastractTOtransfac[key]
    try:
        factorIDS = databaseDict[transfacVersion]
    except:
        factorIDS = []
        for factorNames in transfacVersion:
            factorIDS += databaseDict[factorNames]
    for factorID in factorIDS:
        temporaryInteractionList = [item for item in tfInteractsWith[factorID]]
        resultCheck[key][factorID] = {"TruePositive":0,"FalsePositive":0,"FalseNegative":0}
        trueResults = truthTable_Binding[key]
        for result in trueResults:
            if result in temporaryInteractionList:
                resultCheck[key][factorID]["TruePositive"] += 1
                temporaryInteractionList.remove(result)
            else:
                resultCheck[key][factorID]["FalseNegative"] += 1    
        resultCheck[key][factorID]["FalsePositive"] = len(temporaryInteractionList)


In [10]:
print("\t\t{}\t{}\t{}".format("TP", "FP", "FN"))
for key in resultCheck.keys():
    print(key)
    for factorID in resultCheck[key]:
        TP = resultCheck[key][factorID]["TruePositive"]
        FP = resultCheck[key][factorID]["FalsePositive"]
        FN = resultCheck[key][factorID]["FalseNegative"]
        print("{}\t{}\t{}\t{}".format(factorID, TP, FP, FN))

		TP	FP	FN
Abf1p
F$ABF1_01	8	37	0
F$ABF1_02	0	0	8
F$ABF1_03	8	30	0
F$ABF1_C	8	33	0
F$ABF1_Q6	0	0	8
Ace2p
F$ACE2_01	1	11	1
Adr1p
F$ADR1_01	14	22	3
F$ADR1_02	14	23	3
F$ADR1_Q6	7	9	10
Aft1p
F$RCS1_01	3	27	3
F$RCS1_Q2	2	9	4
Aft2p
F$AFT2_01	1	35	0
Arg80p
F$ARGRI_01	0	27	0
Arg81p
F$ARGRII_01	0	26	0
Aro80p
F$ARO80_01	1	29	0
Arr1p
F$ARR1_01	0	18	0
Ash1p
F$ASH1_01	2	33	1
Azf1p
F$AZF1_01	0	36	0
Bas1p
F$BAS1_01	0	28	0
F$BAS1_Q2	0	23	0
Cat8p
F$CAT8_01	8	35	0
F$CAT8_Q6	8	25	0
F$CAT8_Q6_01	8	26	0
Cbf1p
F$CBF1_01	9	14	3
F$CBF1_02	0	0	12
F$CBF1_03	9	17	3
F$CBF1_04	6	11	6
F$CBF1_05	11	22	1
F$CBF1_B	6	11	6
F$CBF1_Q6	12	29	0
Cha4p
F$CHA4_01	0	50	0
F$CHA4_02	0	49	0
Cin5p
F$CIN5_01	19	10	3
Crz1p
F$CRZ1_01	9	35	1
Cst6p
F$CST6_01	2	18	1
Cup2p
F$CUP2_01	0	37	0
F$CUP2_Q4	0	43	0
Cup9p
F$CUP9_01	0	6	1
F$CUP9_02	0	17	1
Dal80p
F$DAL80_01	0	15	0
Dal81p
F$DAL81_01	0	0	2
Dal82p
F$DAL82_01	0	33	0
Ecm22p
F$ECM22_01	0	42	0
F$ECM22_02	0	39	0
Ert1p
F$YBR239C_01	6	28	1
F$YBR239C_02	6	27	1
Fhl1p
F$FHL1_01	12	16	7
F$FHL1_02	

In [11]:
lowAccuracyMatrices = set()
with open("Accuracy.txt") as acc:
    for line in acc:
        lineListed = line.split()
        try:
            if float(lineListed[2]) < 0.5:
                lowAccuracyMatrices.add(lineListed[0])
        except:
            continue

lowAccuracyMatrices

{'F$MAC1_Q6', 'F$MAC1_Q6_01', 'F$MATALPHA2_08', 'F$MSN2_Q4'}

In [12]:
bestMatrices = {}
bestDatabaseSet = set()
for key in resultCheck.keys():
    bestFN = 100
    for factorID in resultCheck[key].keys():
        if factorID in lowAccuracyMatrices:
            continue
        elif resultCheck[key][factorID]["FalseNegative"] < bestFN:
            bestMatrix = resultCheck[key][factorID]
            bestKey = factorID
            bestFN = resultCheck[key][factorID]["FalseNegative"]
            bestFP = resultCheck[key][factorID]["FalsePositive"]
        elif resultCheck[key][factorID]["FalseNegative"] == bestFN and resultCheck[key][factorID]["FalsePositive"] <= bestFP:
            bestMatrix = resultCheck[key][factorID]
            bestKey = factorID
            bestFP = resultCheck[key][factorID]["FalsePositive"]
        else:
            continue
    bestMatrices[key] = bestMatrix
    bestDatabaseSet.add(bestKey)
            

In [13]:
totalTP, totalFN = [0, 0]
for key in bestMatrices.keys():
    if key == "Yap5p" or key == "Yap6p" or key == "Spt23p" or key == "Ste12p":
        continue
    totalTP += bestMatrices[key]["TruePositive"]
    totalFN += bestMatrices[key]["FalseNegative"]
totalTP/(totalFN+totalTP)*100

70.59679767103349

In [14]:
bestMatrices

{'Abf1p': {'TruePositive': 8, 'FalsePositive': 30, 'FalseNegative': 0},
 'Ace2p': {'TruePositive': 1, 'FalsePositive': 11, 'FalseNegative': 1},
 'Adr1p': {'TruePositive': 14, 'FalsePositive': 22, 'FalseNegative': 3},
 'Aft1p': {'TruePositive': 3, 'FalsePositive': 27, 'FalseNegative': 3},
 'Aft2p': {'TruePositive': 1, 'FalsePositive': 35, 'FalseNegative': 0},
 'Arg80p': {'TruePositive': 0, 'FalsePositive': 27, 'FalseNegative': 0},
 'Arg81p': {'TruePositive': 0, 'FalsePositive': 26, 'FalseNegative': 0},
 'Aro80p': {'TruePositive': 1, 'FalsePositive': 29, 'FalseNegative': 0},
 'Arr1p': {'TruePositive': 0, 'FalsePositive': 18, 'FalseNegative': 0},
 'Ash1p': {'TruePositive': 2, 'FalsePositive': 33, 'FalseNegative': 1},
 'Azf1p': {'TruePositive': 0, 'FalsePositive': 36, 'FalseNegative': 0},
 'Bas1p': {'TruePositive': 0, 'FalsePositive': 23, 'FalseNegative': 0},
 'Cat8p': {'TruePositive': 8, 'FalsePositive': 25, 'FalseNegative': 0},
 'Cbf1p': {'TruePositive': 12, 'FalsePositive': 29, 'FalseNe

In [15]:
totalLargeTP, totalLargeFN = [0,0]
for key in bestMatrices.keys():
    if key == "Yap5p" or key == "Yap6p" or key == "Spt23p" or key == "Ste12p":
        continue;
    if bestMatrices[key]["TruePositive"] + bestMatrices[key]["FalseNegative"] >= 10:
        print("{}: {}".format(key,bestMatrices[key]))
        totalLargeTP += bestMatrices[key]["TruePositive"]
        totalLargeFN += bestMatrices[key]["FalseNegative"]
        print(bestMatrices[key]["TruePositive"]/(bestMatrices[key]["TruePositive"]+bestMatrices[key]["FalseNegative"])*100)
print(totalLargeTP/(totalLargeFN+totalLargeTP)*100)

Adr1p: {'TruePositive': 14, 'FalsePositive': 22, 'FalseNegative': 3}
82.35294117647058
Cbf1p: {'TruePositive': 12, 'FalsePositive': 29, 'FalseNegative': 0}
100.0
Cin5p: {'TruePositive': 19, 'FalsePositive': 10, 'FalseNegative': 3}
86.36363636363636
Crz1p: {'TruePositive': 9, 'FalsePositive': 35, 'FalseNegative': 1}
90.0
Fhl1p: {'TruePositive': 12, 'FalsePositive': 13, 'FalseNegative': 7}
63.1578947368421
Fkh1p: {'TruePositive': 21, 'FalsePositive': 9, 'FalseNegative': 10}
67.74193548387096
Fkh2p: {'TruePositive': 18, 'FalsePositive': 14, 'FalseNegative': 10}
64.28571428571429
Gcn4p: {'TruePositive': 10, 'FalsePositive': 19, 'FalseNegative': 2}
83.33333333333334
Gcr1p: {'TruePositive': 10, 'FalsePositive': 21, 'FalseNegative': 0}
100.0
Gcr2p: {'TruePositive': 8, 'FalsePositive': 26, 'FalseNegative': 4}
66.66666666666666
Hap1p: {'TruePositive': 6, 'FalsePositive': 28, 'FalseNegative': 4}
60.0
Ino2p: {'TruePositive': 14, 'FalsePositive': 18, 'FalseNegative': 7}
66.66666666666666
Ino4p: {'

In [16]:
bestDatabaseSet

{'F$ABF1_03',
 'F$ACE2_01',
 'F$ADR1_01',
 'F$AFT2_01',
 'F$ARGRII_01',
 'F$ARGRI_01',
 'F$ARO80_01',
 'F$ARR1_01',
 'F$ASH1_01',
 'F$AZF1_01',
 'F$BAS1_Q2',
 'F$CAT8_Q6',
 'F$CBF1_Q6',
 'F$CHA4_02',
 'F$CIN5_01',
 'F$CRZ1_01',
 'F$CST6_01',
 'F$CUP2_01',
 'F$CUP9_01',
 'F$DAL80_01',
 'F$DAL81_01',
 'F$DAL82_01',
 'F$ECM22_02',
 'F$FHL1_02',
 'F$FKH1_01',
 'F$FKH2P_01',
 'F$FZF1_01',
 'F$GAL4_02',
 'F$GAT1_02',
 'F$GCN4_Q2',
 'F$GCR1_01',
 'F$GCR2_01',
 'F$GIS1_01',
 'F$GLN3_01',
 'F$GSM1_01',
 'F$GZF3_01',
 'F$HAC1_Q2',
 'F$HAL9_02',
 'F$HAP1_B',
 'F$HAP234_01',
 'F$HCM1P_02',
 'F$HSF_Q6',
 'F$IFH1_01',
 'F$IME1_01',
 'F$INO2_01',
 'F$INO4_01',
 'F$IXR1_01',
 'F$LEU3_B',
 'F$LYS14_02',
 'F$MAL63_Q5',
 'F$MBP1_01',
 'F$MCM1_01',
 'F$MET31_01',
 'F$MET32_02',
 'F$MET4_01',
 'F$MGA1_01',
 'F$MIG1_03',
 'F$MIG2_01',
 'F$MIG3_02',
 'F$MOT2_01',
 'F$MOT3_01',
 'F$MSN2_01',
 'F$MSN4_01',
 'F$NDT80_01',
 'F$NRG1_01',
 'F$PDR1_02',
 'F$PDR3_02',
 'F$PDR8_01',
 'F$PHD1_01',
 'F$PHO2_02',
 'F$PH